# 测试 Better-Adapt 策略在 NBC 指标下对比 ADAPT 的表现

In [ ]:
import sys
sys.path.append('/mnt/e/_01Experimental_task/Graduation_Project/better-adapt')

In [ ]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        print("yes!")
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)

## 加载模型

### 1.分割数据集

In [ ]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

设置时间 

In [ ]:
from datetime import datetime
#from concurrent.futures import ThreadPoolExecutor # 多线程尝试
from tqdm import tqdm

# 设置时间  
set_hours=0
set_minutes=1
set_seconds=0

选择样本

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import random

# 超参数 选取的样本数量(至少2个)
N = 5
M = len(x_test)

In [ ]:
# 挑选一些图片用于白盒测试
# indices = [800, 3433, 3496, 3592, 4322, 4395, 4706, 6101, 6223, 6715, 
#            7190, 7302, 7491, 7715, 8149, 8267, 8608, 9197, 9721, 9795]

# 随机取样
indices = random.sample(range(0, M + 1), N)
print(indices)

In [ ]:
images = x_test[indices]
fig, ax = plt.subplots(1, len(images), figsize=(len(images) * 2, 2))
for i, idx in enumerate(indices):
    ax[i].set_axis_off()
    ax[i].title.set_text(str(np.argmax(y_test[idx])))
    ax[i].imshow(np.reshape(images[i], (28, 28)), cmap='gray')
plt.show()

### 2. 导入模型

In [ ]:
from model.model import LeNet5

model = LeNet5()
model.summary()

### 3. 训练

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

_ = model.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=256, epochs=10, verbose=1)

The final accuracy of the trained model is about 99%.

## 白盒测试

### 2. 创建测试策略

In [ ]:
from adapt.network import Network
from adapt.fuzzer import WhiteBoxFuzzer

In [ ]:
network = Network(model)

In [ ]:
import importlib
import adapt.metric as me
importlib.reload(me)

### 选择覆盖率指标

In [ ]:
from adapt.metric import NeuronCoverage
from adapt.metric import NeuronBoundaryCoverage
from adapt.metric import StrongNeuronActivationCoverage
from adapt.metric import SignSignCoverage

NC = NeuronCoverage()
NBC = NeuronBoundaryCoverage()
SNAC = StrongNeuronActivationCoverage()
SSC = SignSignCoverage()

### 加载输入到测试工具

In [ ]:
from adapt.strategy import BetterAdaptiveStrategy
from adapt.strategy import AdaptiveParameterizedStrategy

## NBC

In [ ]:
fuzzers_better_nbc = []

In [ ]:
for image in images:
    strategy = BetterAdaptiveStrategy(network)
    fuzzer = WhiteBoxFuzzer(network, image, NBC, strategy)
    fuzzers_better_nbc.append(fuzzer)

In [ ]:
archives_better_nbc = []
for i in tqdm(range(len(fuzzers_better_nbc)), desc='Better-ADAPT running...', unit='fuzzer'):  # 使用 tqdm 创建进度条
    fuzzer = fuzzers_better_nbc[i]
    result = fuzzer.start(hours=set_hours, minutes=set_minutes, seconds=set_seconds, append='min_dist')
    archives_better_nbc.append(result)
print('Done')

### 4. 测试结果

In [ ]:
def align_coverages(coverages, target_length=61):
    aligned_coverages = []
    for coverage in coverages:
        if len(coverage) > target_length:
            aligned_coverages.append(coverage[:target_length])  # 截断
        else:
            aligned_coverages.append(coverage + [coverage[-1]] * (target_length - len(coverage)))  # 填充
    return np.array(aligned_coverages)

In [ ]:
# 这里代表横坐标的时间应与设置的时间相同（当然也可以自己调整）
set_time = set_hours*3600+set_minutes*60+set_seconds+1

In [ ]:
times = list(range(set_time))


coverages_better = []
coverages = []
for archive in archives_better_nbc:
    current_t = 0
    coverage = []
    for t, cov in archive.timestamp:
        if t > current_t:
            coverage.append(cov)
            current_t += 1
    coverages.append(coverage)
coverages = align_coverages(coverages, set_time)
coverages = np.array(coverages, dtype=object)
coverages = np.mean(coverages, axis=0)
coverages_better.append(coverages)


In [ ]:

# 绘制 better 的折线图
plt.plot(coverages_better[i], linestyle='-', label=f'Better-NBC')

# 添加图例、标题和坐标轴标签
plt.legend(title="Legend", loc="upper left", fontsize=10)  # 设置图例标题和位置
plt.title('LeNet-5')
plt.xlabel('Time/s')
plt.ylabel('Coverage')

# 显示图形
plt.show()

In [ ]:
B = [[],[],[]]
for archives in archives_better_nbc:
    total = 0
    adversarial = 0
    distance = 0
    for archive in archives: # 具体到每个样本的测试输出
        total += archive.total
        adversarial += archive.adversarials
        d = 0.5 if archive.adversarials == 0 else np.mean(np.concatenate([archive.distance[label] for label in archive.distance.keys() if label != archive.label]))
        distance += d
    
    B[0].append(total)
    B[1].append(adversarial)
    B[2].append(distance / len(archives))

In [ ]:
print("Better-ADAPT output:")
print("totals:", B[0])
print("adversarials:", B[1])
print("A/T:", np.array(B[1])/np.array(B[0]))
print("distance:", B[2])

### 选择打印样本的对应结果

In [ ]:
print(f'Better-ADAPT: Metric is NBC')
for index, archive in enumerate(archives_better_nbc):
    print(f"第{index}个输入样本")
    num_subplots = len(archive.found_labels)

    # 创建子图
    fig, ax = plt.subplots(1, num_subplots, figsize=(num_subplots * 2, 2))

    # 如果只有一个子图，将 ax 转换为列表
    if num_subplots == 1:
        ax = [ax]

    # 绘制每个子图
    for i, label in enumerate(archive.found_labels.keys()):
        ax[i].set_axis_off()  # 关闭坐标轴
        ax[i].set_title(str(label))  # 设置标题
        ax[i].imshow(
            np.reshape(archive.inputs[label][0], (28, 28)).clip(0, 1),
            cmap='gray'
        )

    plt.show()